# Create realisations from MAGICC output for a specific ESM and MESMER

This uses data from the [RCMIP Phase 1 paper](https://doi.org/10.5194/gmd-13-5175-2020), archived [here](http://doi.org/10.5281/zenodo.4016613) (it's a pain to download the whole thing so I've just extracted a couple of timeseries of interest as illustration.

In [1]:
import joblib
import os.path

import mesmer.create_emulations
import scmdata
import xarray as xr
from openscm_units import unit_registry

<IPython.core.display.Javascript object>

## Configuration

In [2]:
SCENARIOS_TO_RUN = ["ssp*", "rcp*"]
N_REALISATIONS_PER_SCENARIO = 5
YEARS_TO_EMULATE = range(1850, 2150 + 1)
OUT_FILE = "canesm5_example_emulation.nc"

## Load data

### MAGICC output

In [3]:
magicc_output_dir = os.path.join(
    ".",
    "rcmip-phase-1-archive",
)

magicc_output_gsat_file = os.path.join(
    magicc_output_dir,
    "rcmip-phase-1_magicc7.1.0.beta-canesm5-r1i1p1f1_world_surface-air-temperature-change.csv",
)
display(magicc_output_gsat_file)

magicc_output_ohc_file = os.path.join(
    magicc_output_dir,
    "rcmip-phase-1_magicc7.1.0.beta-canesm5-r1i1p1f1_world_heat-content-ocean.csv",
)
display(magicc_output_ohc_file)

'./rcmip-phase-1-archive/rcmip-phase-1_magicc7.1.0.beta-canesm5-r1i1p1f1_world_surface-air-temperature-change.csv'

'./rcmip-phase-1-archive/rcmip-phase-1_magicc7.1.0.beta-canesm5-r1i1p1f1_world_heat-content-ocean.csv'

In [4]:
magicc_gsat_output = scmdata.ScmRun(magicc_output_gsat_file).filter(
    scenario=SCENARIOS_TO_RUN
)
display(magicc_gsat_output)

magicc_ohc_output = scmdata.ScmRun(magicc_output_ohc_file).filter(
    scenario=SCENARIOS_TO_RUN
)
display(magicc_ohc_output)

<scmdata.ScmRun (timeseries: 14, timepoints: 651)>
Time:
	Start: 1850-01-01T00:00:00
	End: 2500-01-01T00:00:00
Meta:
	                         climatemodel   model region  \
	39  MAGICC7.1.0.beta_CANESM5_R1I1P1F1  an_iam  World   
	40  MAGICC7.1.0.beta_CANESM5_R1I1P1F1  an_iam  World   
	41  MAGICC7.1.0.beta_CANESM5_R1I1P1F1  an_iam  World   
	42  MAGICC7.1.0.beta_CANESM5_R1I1P1F1  an_iam  World   
	43  MAGICC7.1.0.beta_CANESM5_R1I1P1F1  an_iam  World   
	44  MAGICC7.1.0.beta_CANESM5_R1I1P1F1  an_iam  World   
	45  MAGICC7.1.0.beta_CANESM5_R1I1P1F1  an_iam  World   
	46  MAGICC7.1.0.beta_CANESM5_R1I1P1F1  an_iam  World   
	47  MAGICC7.1.0.beta_CANESM5_R1I1P1F1  an_iam  World   
	48  MAGICC7.1.0.beta_CANESM5_R1I1P1F1  an_iam  World   
	49  MAGICC7.1.0.beta_CANESM5_R1I1P1F1  an_iam  World   
	50  MAGICC7.1.0.beta_CANESM5_R1I1P1F1  an_iam  World   
	51  MAGICC7.1.0.beta_CANESM5_R1I1P1F1  an_iam  World   
	52  MAGICC7.1.0.beta_CANESM5_R1I1P1F1  an_iam  World   
	
	                     scen

<scmdata.ScmRun (timeseries: 14, timepoints: 651)>
Time:
	Start: 1850-01-01T00:00:00
	End: 2500-01-01T00:00:00
Meta:
	                         climatemodel   model region  \
	39  MAGICC7.1.0.beta_CANESM5_R1I1P1F1  an_iam  World   
	40  MAGICC7.1.0.beta_CANESM5_R1I1P1F1  an_iam  World   
	41  MAGICC7.1.0.beta_CANESM5_R1I1P1F1  an_iam  World   
	42  MAGICC7.1.0.beta_CANESM5_R1I1P1F1  an_iam  World   
	43  MAGICC7.1.0.beta_CANESM5_R1I1P1F1  an_iam  World   
	44  MAGICC7.1.0.beta_CANESM5_R1I1P1F1  an_iam  World   
	45  MAGICC7.1.0.beta_CANESM5_R1I1P1F1  an_iam  World   
	46  MAGICC7.1.0.beta_CANESM5_R1I1P1F1  an_iam  World   
	47  MAGICC7.1.0.beta_CANESM5_R1I1P1F1  an_iam  World   
	48  MAGICC7.1.0.beta_CANESM5_R1I1P1F1  an_iam  World   
	49  MAGICC7.1.0.beta_CANESM5_R1I1P1F1  an_iam  World   
	50  MAGICC7.1.0.beta_CANESM5_R1I1P1F1  an_iam  World   
	51  MAGICC7.1.0.beta_CANESM5_R1I1P1F1  an_iam  World   
	52  MAGICC7.1.0.beta_CANESM5_R1I1P1F1  an_iam  World   
	
	                     scen

#### Convert ocean heat content to ocean heat uptake

In [5]:
earth_surface_area = 510.1 * 10 ** 6 * unit_registry("km^2")
magicc_hfds_output = (
    magicc_ohc_output.delta_per_delta_time(out_var="Heat Uptake|Ocean")
    .convert_unit("W")
) / earth_surface_area
magicc_hfds_output

<scmdata.ScmRun (timeseries: 14, timepoints: 650)>
Time:
	Start: 1850-07-02T12:00:00
	End: 2499-07-02T12:00:00
Meta:
	                         climatemodel   model region  \
	0   MAGICC7.1.0.beta_CANESM5_R1I1P1F1  an_iam  World   
	1   MAGICC7.1.0.beta_CANESM5_R1I1P1F1  an_iam  World   
	2   MAGICC7.1.0.beta_CANESM5_R1I1P1F1  an_iam  World   
	3   MAGICC7.1.0.beta_CANESM5_R1I1P1F1  an_iam  World   
	4   MAGICC7.1.0.beta_CANESM5_R1I1P1F1  an_iam  World   
	5   MAGICC7.1.0.beta_CANESM5_R1I1P1F1  an_iam  World   
	6   MAGICC7.1.0.beta_CANESM5_R1I1P1F1  an_iam  World   
	7   MAGICC7.1.0.beta_CANESM5_R1I1P1F1  an_iam  World   
	8   MAGICC7.1.0.beta_CANESM5_R1I1P1F1  an_iam  World   
	9   MAGICC7.1.0.beta_CANESM5_R1I1P1F1  an_iam  World   
	10  MAGICC7.1.0.beta_CANESM5_R1I1P1F1  an_iam  World   
	11  MAGICC7.1.0.beta_CANESM5_R1I1P1F1  an_iam  World   
	12  MAGICC7.1.0.beta_CANESM5_R1I1P1F1  an_iam  World   
	13  MAGICC7.1.0.beta_CANESM5_R1I1P1F1  an_iam  World   
	
	                     scen

### MESMER configuration

In [6]:
# TODO: use CanESM5 specific calibrations rather than IPSL
# @lea you can use whatever methods you want to get params_lt, params_lv, params_gv,
# seeds, land_fractions here, these are just for testing
mesmer_bundle_file = os.path.join(
    "..", "tests", "test-data", "mesmer-bundles", "test-generic-mesmer-bundle.pkl"
)
mesmer_bundle = joblib.load(mesmer_bundle_file)

params_lt = mesmer_bundle["params_lt"]
params_lv = mesmer_bundle["params_lv"]
params_gv_T = mesmer_bundle["params_gv_T"]
seeds = mesmer_bundle["seeds"]
land_fractions = mesmer_bundle["land_fractions"]

## Define run function

Given it's only 150 lines, doing this in the notebook makes it slightly easier to prototype, experiment with parellelisation etc.

In [7]:
def _draw_realisations_from_mesmer_config_and_openscm_output(
    params_lt,
    params_lv,
    params_gv_T,
    seeds,
    land_fractions,
    openscm_gsat,
    openscm_hfds=None,
    n_realisations_per_scenario=5,
):
    """
    Parameters
    ----------
    params_lt : todo
        TODO: description of minimal amount of information required
        
    params_lv : todo
        TODO: description of minimal amount of information required
        
    params_gv_T : todo
        TODO: description of minimal amount of information required
        
    seeds : todo
        TODO: description of minimal amount of information required
        
    land_fractions : todo
        TODO: description of minimal amount of information required
        
    mesmer_bundle_file : str
        File in which the MESMER bundle is saved

    openscm_gsat : :obj:`scmdata.ScmRun`
        Global-mean surface air temperature change output to be used to drive MESMER's emulations

    openscm_hfds : [None, :obj:`scmdata.ScmRun`]
        If supplied, global-mean ocean heat uptake output to be used to drive MESMER's emulations

    n_realisations_per_scenario : int
        Number of realisations to draw for each scenario in ``openscm_gsat`` (and ``openscm_hfds`` if supplied)

    Returns
    -------
    :obj:`xr.Dataset`
        Emulations for each scenario

    Raises
    ------
    ValueError
        The scenarios in `openscm_gsat`` and ``openscm_hfds`` (if supplied) aren't the same

    ValueError
        [TODO talk to Lea about what to do here] The scenarios in ``openscm_gsat`` don't match those for which MESMER has been calibrated

    ValueError
        [TODO talk to Lea about what to do here] The scenarios in ``openscm_gsat`` span a longer time than MESMER has been calibrated for

    ValueError
        ``openscm_gsat`` contains variables other than ``"Surface Air Temperature Change"``

    ValueError
        ``openscm_hfds`` contains variables other than ``"Heat Uptake|Ocean"``
    """
    gsat_scenarios = set(openscm_gsat.get_unique_meta("scenario"))
    hfds_scenarios = set(openscm_hfds.get_unique_meta("scenario"))
    if gsat_scenarios != hfds_scenarios:
        raise ValueError(
            "gsat_scenarios: {}, hfds_scenarios: {}".format(
                gsat_scenarios, hfds_scenarios
            )
        )

    openscm_gsat_for_mesmer = _prepare_openscm_gsat(openscm_gsat)
    openscm_hfds_for_mesmer = _prepare_openscm_hfds(openscm_hfds)

    hard_coded_hist_years = range(1850, 2014 + 1)
    hard_coded_scen_years = range(2015, 3000 + 1)

    out = []
    for scen_gsat in openscm_gsat_for_mesmer.groupby("scenario"):
        scenario = scen_gsat.get_unique_meta("scenario", True)
        # if scenario not in mesmer_bundle["time"]:
        #     raise KeyError("No MESMER calibration available for: {}".format(scenario))

        hist_tas = _filter_and_assert_1d(scen_gsat.filter(year=hard_coded_hist_years))
        scen_tas = _filter_and_assert_1d(scen_gsat.filter(year=hard_coded_scen_years))

        openscm_hfds_for_mesmer_scen = openscm_hfds_for_mesmer.filter(scenario=scenario)
        hist_hfds = _filter_and_assert_1d(openscm_hfds_for_mesmer_scen.filter(year=hard_coded_hist_years))
        scen_hfds = _filter_and_assert_1d(openscm_hfds_for_mesmer_scen.filter(year=hard_coded_scen_years))

        # TODO: remove hard-coding and actually map up with what MESMER expects
        preds_lt_scenario = {}
        for predictor in mesmer_bundle["params_lt"]["preds"]:
            if predictor == "gttas":
                hist_vals = hist_tas
                scen_vals = scen_tas
            elif predictor == "gttas2":
                hist_vals = hist_tas ** 2
                scen_vals = scen_tas ** 2
            elif predictor == "gthfds":
                hist_vals = hist_hfds
                scen_vals = scen_hfds

            preds_lt_scenario[predictor] = {"hist": hist_vals, scenario: scen_vals}

        result_scenario = mesmer.create_emulations.make_realisations(
            preds_lt=preds_lt_scenario,
            params_lt=params_lt,
            params_lv=params_lv,
            params_gv_T=params_gv_T,
            n_realisations=n_realisations_per_scenario,
            seeds=seeds,
            land_fractions=land_fractions,
            time={
                "hist": scen_gsat.filter(year=hard_coded_hist_years)["year"].values,
                scenario: scen_gsat.filter(year=hard_coded_scen_years)["year"].values
            },
        )

        result_scenario = (
            result_scenario
            .squeeze(dim="scenario", drop=True)
            .expand_dims({"scenario": [scenario]})
        )
        out.append(result_scenario)

    out = xr.merge(out)

    return out


def _prepare_openscm_gsat(openscm_gsat):
    return openscm_gsat.convert_unit("K")


def _prepare_openscm_hfds(openscm_hfds):
    return openscm_hfds.convert_unit("W/m^2")


def _filter_and_assert_1d(scmrun, **kwargs):
    filtered = scmrun.filter(**kwargs).values.squeeze()
    if len(filtered.shape) != 1:
        raise ValueError(
            "Filters gave non-1D data: {}, {}".format(filtered.shape, kwargs)
        )

    return filtered


In [8]:
# TODO: make all MESMER's print output controllable
result = _draw_realisations_from_mesmer_config_and_openscm_output(
    params_lt,
    params_lv,
    params_gv_T,
    seeds,
    land_fractions,
    openscm_gsat=magicc_gsat_output.filter(year=YEARS_TO_EMULATE),
    openscm_hfds=magicc_hfds_output.filter(year=YEARS_TO_EMULATE),
    n_realisations_per_scenario=N_REALISATIONS_PER_SCENARIO,
)

result

Start with OLS
Start with AR(1) with spatially correlated innovations.
Draw the innovations
Compute the contribution to emus_lv by the AR(1) process with the spatially correlated innovations
Create the full local variability emulations
Start with OLS
Start with AR(1) with spatially correlated innovations.
Draw the innovations
Compute the contribution to emus_lv by the AR(1) process with the spatially correlated innovations
Create the full local variability emulations
Start with OLS
Start with AR(1) with spatially correlated innovations.
Draw the innovations
Compute the contribution to emus_lv by the AR(1) process with the spatially correlated innovations
Create the full local variability emulations
Start with OLS
Start with AR(1) with spatially correlated innovations.
Draw the innovations
Compute the contribution to emus_lv by the AR(1) process with the spatially correlated innovations
Create the full local variability emulations
Start with OLS
Start with AR(1) with spatially correlate

<xarray.Dataset>
Dimensions:      (realisation: 5, scenario: 14, year: 301, z: 117)
Coordinates:
  * scenario     (scenario) <U25 'rcp26' 'rcp45' ... 'ssp534-over' 'ssp585'
  * realisation  (realisation) int64 0 1 2 3 4
  * year         (year) int64 1850 1851 1852 1853 1854 ... 2147 2148 2149 2150
    lat          (z) float64 -49.5 -40.5 -31.5 -31.5 ... 76.5 76.5 76.5 76.5
    lon          (z) float64 -72.0 -72.0 -72.0 -54.0 ... -36.0 -18.0 90.0 108.0
Dimensions without coordinates: z
Data variables:
    tas          (scenario, realisation, year, z) float64 0.6578 ... 65.34

In [9]:
result.to_netcdf(OUT_FILE)